In [17]:
import threading
import time

class EventualConsistentModel:
    def __init__(self):
        self.parameters = {}  # Initialize an empty dictionary to store parameters
        self.lock = threading.Lock()  # Create a lock for synchronization

    def update_parameters(self, params):
        with self.lock:  # Acquire the lock to ensure thread safety
            self.parameters.update(params)  # Update the parameters with the given values
            print(f"Parameters updated to: {self.parameters}")  # Print the updated parameters

    def get_parameters(self):
        with self.lock:  # Acquire the lock to ensure thread safety
            print(f"Parameters read: {self.parameters}")  # Print the current parameters
            return self.parameters  # Return the current parameters

# Function to simulate asynchronous update propagation
def simulate_async_propagation(model, params, delay, event):
    time.sleep(delay)  # Simulate delay in propagation
    model.update_parameters(params)  # Update the model parameters after the delay
    event.set()  # Signal that the update is complete

# Usage
model = EventualConsistentModel()

def worker1(event):
    model.get_parameters()  # Print the initial empty parameters
    # Start a thread to simulate asynchronous update with a delay of 2 seconds
    threading.Thread(target=simulate_async_propagation, args=(model, {'weight': 0.5}, 2, event)).start()
    #A new thread is created with the target function simulate_async_propagation.
    #The arguments to be passed to the function are provided as a tuple: (model, {'weight': 0.5}, 2, event).

def worker2(event):
    time.sleep(1)  # Ensure this print happens before the update completes
    model.get_parameters()  # Read the parameters before they are updated
    event.wait()  # Wait until the update is complete
    model.get_parameters()  # Read the parameters after the update

# Create an event for synchronization
update_event = threading.Event()

# Create and start threads for workers
threads = [threading.Thread(target=worker1, args=(update_event,)), threading.Thread(target=worker2, args=(update_event,))]
#This line of code creates a list of two thread objects, each assigned to execute a specific worker function with the provided arguments.
#threading.Thread(target=worker1, args=(update_event,))
#threading.Thread: Creates a new thread object.
#target=worker1: Specifies that the worker1 function will be the entry point of this thread.
#args=(update_event,): Passes the argument update_event to the worker1 function. The comma in the tuple is necessary to indicate a single-element tuple.


for t in threads:
    t.start()

for t in threads:
    t.join()  # Wait for all threads to complete


Parameters read: {}
Parameters read: {}
Parameters updated to: {'weight': 0.5}
Parameters read: {'weight': 0.5}


In [18]:
import threading  # Import threading module to handle multiple threads
import time  # Import time module to simulate delays

class EventualConsistentModel:
    def __init__(self):
        self.parameters = {}  # Initialize an empty dictionary to store parameters
        self.lock = threading.Lock()  # Create a lock for synchronization

    def update_parameters(self, params):
        with self.lock:  # Acquire the lock to ensure thread safety
            self.parameters.update(params)  # Update the parameters with the given values
            print(f"Parameters updated to: {self.parameters}")  # Print the updated parameters

    def get_parameters(self):
        with self.lock:  # Acquire the lock to ensure thread safety
            print(f"Parameters read: {self.parameters}")  # Print the current parameters
            return self.parameters  # Return the current parameters

# Function to simulate asynchronous update propagation
def simulate_async_propagation(model, params, delay, event):
    time.sleep(delay)  # Simulate delay in propagation
    model.update_parameters(params)  # Update the model parameters after the delay
    event.set()  # Signal that the update is complete

# Usage
model = EventualConsistentModel()  # Create an instance of the EventualConsistentModel

def worker1(event):
    model.get_parameters()  # Print the initial empty parameters
    # Start a thread to simulate asynchronous update with a delay of 2 seconds
    threading.Thread(target=simulate_async_propagation, args=(model, {'weight': 0.5}, 2, event)).start()

def worker2(event):
    time.sleep(1)  # Ensure this print happens before the update completes
    model.get_parameters()  # Read the parameters before they are updated
    event.wait()  # Wait until the update is complete
    model.get_parameters()  # Read the parameters after the update

# Create an event for synchronization
update_event = threading.Event()

# Create and start threads for workers
threads = [threading.Thread(target=worker1, args=(update_event,)), threading.Thread(target=worker2, args=(update_event,))]

for t in threads:
    t.start()  # Start each thread

for t in threads:
    t.join()  # Wait for all threads to complete


Parameters read: {}
Parameters read: {}
Parameters updated to: {'weight': 0.5}
Parameters read: {'weight': 0.5}


Strong Consistency

In [19]:
import threading

class StrongConsistentModel:
    def __init__(self):
        self.parameters = {}  # Initialize an empty dictionary to store parameters
        self.lock = threading.Lock()  # Create a lock for synchronization

    def update_parameters(self, params):
        with self.lock:  # Acquire the lock to ensure thread safety
            self.parameters.update(params)  # Update the parameters with the given values
            print(f"Parameters updated to: {self.parameters}")  # Print the updated parameters

    def get_parameters(self):
        with self.lock:  # Acquire the lock to ensure thread safety
            print(f"Parameters read: {self.parameters}")  # Print the current parameters
            return self.parameters  # Return the current parameters

# Usage
model = StrongConsistentModel()

def writer():
    model.update_parameters({'weight': 0.5})  # Update the parameters with new values

def reader():
    model.get_parameters()  # Read the current parameters

# Create and start threads for writer and reader
threads = [threading.Thread(target=writer), threading.Thread(target=reader)]

for t in threads:
    t.start()

for t in threads:
    t.join()  # Wait for all threads to complete


Parameters updated to: {'weight': 0.5}
Parameters read: {'weight': 0.5}


Strong Consistency

In [20]:
import threading
import time

class StrongConsistentModel:
    def __init__(self):
        self.parameters = {}  # Initialize an empty dictionary to store parameters
        self.lock = threading.Lock()  # Create a lock for synchronization

    def update_parameters(self, params, thread_id):
        with self.lock:  # Acquire the lock to ensure thread safety
            self.parameters.update(params)  # Update the parameters with the given values
            print(f"[{time.strftime('%H:%M:%S')}] Thread {thread_id} updated parameters to: {self.parameters}")  # Print the updated parameters

    def get_parameters(self, thread_id):
        with self.lock:  # Acquire the lock to ensure thread safety
            print(f"[{time.strftime('%H:%M:%S')}] Thread {thread_id} read parameters: {self.parameters}")  # Print the current parameters
            return self.parameters  # Return the current parameters

# Usage
model = StrongConsistentModel()

# Barrier to synchronize threads
barrier = threading.Barrier(6)  # 1 writer + 5 readers

def writer(thread_id):
    model.update_parameters({'weight': 0.5}, thread_id)  # Update the parameters with new values
    barrier.wait()  # Wait for all threads to reach this point

def reader(thread_id):
    barrier.wait()  # Wait for all threads to reach this point
    model.get_parameters(thread_id)  # Read the current parameters

# Create and start threads for one writer and multiple readers
threads = []
threads.append(threading.Thread(target=writer, args=(1,)))

for i in range(5):
    t_reader = threading.Thread(target=reader, args=(i+2,))
    threads.append(t_reader)

for t in threads:
    t.start()

for t in threads:
    t.join()  # Wait for all threads to complete


[09:55:09] Thread 1 updated parameters to: {'weight': 0.5}
[09:55:09] Thread 6 read parameters: {'weight': 0.5}
[09:55:09] Thread 2 read parameters: {'weight': 0.5}
[09:55:09] Thread 3 read parameters: {'weight': 0.5}
[09:55:09] Thread 4 read parameters: {'weight': 0.5}
[09:55:09] Thread 5 read parameters: {'weight': 0.5}


In [21]:
import threading
import time

class StrongConsistentModel:
    def __init__(self):
        self.parameters = {}  # Initialize an empty dictionary to store parameters
        self.lock = threading.Lock()  # Create a lock for synchronization
        self.condition = threading.Condition(self.lock)  # Create a condition variable associated with the lock
        self.update_count = 0  # Counter to keep track of the number of updates

    def update_parameters(self, params, thread_id):
        with self.condition:  # Acquire the lock associated with the condition variable
            self.parameters.update(params)  # Update the parameters with the given values
            self.update_count += 1  # Increment the update counter
            print(f"[{time.strftime('%H:%M:%S')}] Thread {thread_id} updated parameters to: {self.parameters}")
            self.condition.notify_all()  # Notify all waiting threads that the parameters have been updated

    def get_parameters(self, thread_id):
        with self.condition:  # Acquire the lock associated with the condition variable
            print(f"[{time.strftime('%H:%M:%S')}] Thread {thread_id} read parameters: {self.parameters}")
            return self.parameters  # Return the current parameters

# Usage
model = StrongConsistentModel()

num_writers = 3
num_readers = 5
total_updates = num_writers  # Total number of updates to be performed

def writer(thread_id):
    # Each writer updates the parameters with its thread_id as the value
    model.update_parameters({'weight': thread_id * 0.5}, thread_id)
    with model.condition:
        while model.update_count < total_updates:
            model.condition.wait()  # Wait until all updates are done
    with model.lock:
        print(f"[{time.strftime('%H:%M:%S')}] Thread {thread_id} final check: {model.parameters}")

def reader(thread_id):
    with model.condition:
        while model.update_count < total_updates:
            model.condition.wait()  # Wait for each update
            print(f"[{time.strftime('%H:%M:%S')}] Thread {thread_id} sees parameters after update: {model.parameters}")
    with model.lock:
        print(f"[{time.strftime('%H:%M:%S')}] Thread {thread_id} final read: {model.parameters}")

# Create and start threads for multiple writers and readers
threads = []

# Add writer threads
for i in range(num_writers):
    t_writer = threading.Thread(target=writer, args=(i + 1,))
    threads.append(t_writer)

# Add reader threads
for i in range(num_readers):
    t_reader = threading.Thread(target=reader, args=(i + num_writers + 1,))
    threads.append(t_reader)

# Start all threads
for t in threads:
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()


[09:56:34] Thread 1 updated parameters to: {'weight': 0.5}
[09:56:34] Thread 2 updated parameters to: {'weight': 1.0}
[09:56:34] Thread 3 updated parameters to: {'weight': 1.5}
[09:56:34] Thread 3 final check: {'weight': 1.5}
[09:56:34] Thread 2 final check: {'weight': 1.5}
[09:56:34] Thread 1 final check: {'weight': 1.5}
[09:56:34] Thread 4 final read: {'weight': 1.5}
[09:56:34] Thread 5 final read: {'weight': 1.5}
[09:56:34] Thread 6 final read: {'weight': 1.5}
[09:56:34] Thread 7 final read: {'weight': 1.5}
[09:56:34] Thread 8 final read: {'weight': 1.5}


In [16]:
import threading

class CausalConsistentModel:
    def __init__(self):
        self.parameters = {}  # Initialize an empty dictionary to store parameters
        self.dependency_graph = {}  # Initialize an empty dictionary to store dependencies
        self.lock = threading.Lock()  # Create a lock for synchronization

    def update_parameters(self, params, thread_id, dependency=None):
        with self.lock:  # Acquire the lock to ensure thread safety
            # Check if the dependency is met
            if dependency and not self.dependency_graph.get(dependency):
                print(f"[Thread {thread_id}] Dependency {dependency} not met, update postponed.")
                return
            # Update parameters and dependency graph
            self.parameters.update(params)
            self.dependency_graph.update(params)
            print(f"[Thread {thread_id}] Parameters updated to: {self.parameters}")

    def get_parameters(self, thread_id):
        with self.lock:  # Acquire the lock to ensure thread safety
            print(f"[Thread {thread_id}] Parameters read: {self.parameters}")
            return self.parameters  # Return the current parameters

# Usage
model = CausalConsistentModel()  # Create an instance of the CausalConsistentModel

def writer1():
    model.update_parameters({'x': 'a'}, thread_id=1)  # Update parameter x to 'a'

def writer2():
    model.update_parameters({'y': 'b'}, thread_id=2, dependency='x')  # Update parameter y to 'b' with a dependency on x

def writer3():
    model.update_parameters({'z': 'c'}, thread_id=3, dependency='y')  # Update parameter z to 'c' with a dependency on y

def reader(thread_id):
    model.get_parameters(thread_id)  # Read the current parameters
    print(f"[Thread {thread_id}] read parameters: {model.parameters}")  # Print the parameters read by this thread

# Create and start threads for writers and readers
threads = [
    threading.Thread(target=writer1),  # Create a thread for writer1
    threading.Thread(target=writer2),  # Create a thread for writer2
    threading.Thread(target=writer3),  # Create a thread for writer3
    threading.Thread(target=reader, args=(4,)),  # Create a thread for reader1 with thread_id 4
    threading.Thread(target=reader, args=(5,))   # Create a thread for reader2 with thread_id 5
]

for t in threads:
    t.start()  # Start each thread

for t in threads:
    t.join()  # Wait for all threads to complete


[Thread 1] Parameters updated to: {'x': 'a'}
[Thread 2] Parameters updated to: {'x': 'a', 'y': 'b'}
[Thread 3] Parameters updated to: {'x': 'a', 'y': 'b', 'z': 'c'}
[Thread 4] Parameters read: {'x': 'a', 'y': 'b', 'z': 'c'}
[Thread 4] read parameters: {'x': 'a', 'y': 'b', 'z': 'c'}
[Thread 5] Parameters read: {'x': 'a', 'y': 'b', 'z': 'c'}
[Thread 5] read parameters: {'x': 'a', 'y': 'b', 'z': 'c'}
